In [ ]:
# importing required libarires

import os
import googleapiclient.discovery
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import JSON
import json

# **Youtube API**

In [ ]:
# the YouTube API key

api_key = "Your API Key"


# the list of channel_ids

channel_ids = ["UCq-Fj5jknLsUf-MWSy4_brA"] # youtube channel_id

In [ ]:
# YouTube Data API client
# The client object is used to make requests to the YouTube API

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# **Fetching Channel stats**

In [ ]:
def get_channel_stats(youtube , channel_ids):

  all_data = []

  request = youtube.channels().list(
      part="snippet,contentDetails,statistics",
      id= ",".join(channel_ids))
  response = request.execute()

  for item in response["items"]:
        data =     {"channel_name" : item["snippet"]["title"],
                    "channel_published_date" : item["snippet"]["publishedAt"],
                    "subscribers" : int(item["statistics"]["subscriberCount"]),
                    "total_view" : int(item["statistics"]["viewCount"]),
                    "total_video" : int(item["statistics"]["videoCount"]),
                    "playlist_id" : item["contentDetails"]["relatedPlaylists"]["uploads"] }
        all_data.append(data)

  return all_data

In [ ]:
get_channel_stats(youtube , channel_ids)

[{'channel_name': 'T-Series',
  'channel_published_date': '2006-03-13T14:27:05Z',
  'subscribers': 247000000,
  'total_view': 230876942178,
  'total_video': 19336,
  'playlist_id': 'UUq-Fj5jknLsUf-MWSy4_brA'}]

# **Fetching Video Ids**

In [ ]:
playlist_id = "UUq-Fj5jknLsUf-MWSy4_brA"

In [ ]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
    )

    response = request.execute()

    for item in response["items"]:
        video_ids.append(item["contentDetails"]["videoId"])

    next_page_token = response.get("nextPageToken")  # to check if there are more pages of video IDs to fetch using "nextPageToken"

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )

        response = request.execute()

        for item in response["items"]:
            video_ids.append(item["contentDetails"]["videoId"])

        next_page_token = response.get("nextPageToken")

    return video_ids


In [ ]:
video_ids = get_video_ids(youtube, playlist_id)

In [ ]:
len(video_ids)

19345

# **Fetching the video details from video_ids**

In [ ]:
def get_vid_details(youtube, video_ids):
    all_details = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response["items"]:
            details = {
                "video_id": video["id"],
                "channelTitle": video["snippet"]["channelTitle"],
                "title": video["snippet"]["title"],
                "description": video["snippet"]["description"],
                "tags": video["snippet"].get("tags", []),
                "publishedAt": video["snippet"]["publishedAt"],
                "viewCount": video["statistics"]["viewCount"],
                "likeCount": video["statistics"].get("likeCount", 0),
                "favoriteCount": video["statistics"]["favoriteCount"],
                "commentCount": video["statistics"].get("commentCount", 0),
                "duration": video["contentDetails"]["duration"],
                "definition": video["contentDetails"]["definition"],
                "caption": video["contentDetails"]["caption"]
            }
            all_details.append(details)

    return all_details

df1 = pd.DataFrame(get_vid_details(youtube, video_ids))


In [ ]:
df1 = pd.DataFrame(get_vid_details(youtube, video_ids))

df1

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption
0,F44TK5EHCRo,T-Series,RAFTA RAFTA (Lo-Fi Mix) | Dj Moody | KK | Jeet...,"Presenting the song ""RAFTA RAFTA (Lo-Fi Mix)"" ...","[hindi songs 2023, hindi songs new, bollywood ...",2023-08-20T12:30:08Z,7299,241,0,59,PT4M42S,hd,false
1,adx8Rsjp-c0,T-Series,"TUM HO MERA PYAR (Lo-Fi Mix) | Dj Moody | KK, ...","Presenting the song ""TUM HO MERA PYAR (Lo-Fi M...","[hindi songs 2023, hindi songs new, bollywood ...",2023-08-20T10:30:01Z,4212,144,0,32,PT5M27S,hd,false
2,36nQLyrR7Sw,T-Series,ISHQ HOTHON SE (Lo-Fi Mix) (Audio) | Dj Moody ...,"Presenting the song ""ISHQ HOTHON SE (Lo-Fi Mix...","[hindi songs 2023, hindi songs new, bollywood ...",2023-08-20T09:30:00Z,2849,79,0,31,PT6M10S,hd,false
3,TKbifi5AMQo,T-Series,HUMKO PYAR HUA (Lo-Fi Mix) (Audio) | Dj Moody ...,"Presenting the song ""HUMKO PYAR HUA (Lo-Fi Mix...","[hindi songs 2023, hindi songs new, bollywood ...",2023-08-20T07:30:05Z,3991,116,0,36,PT5M59S,hd,false
4,NNA21SoGqss,T-Series,Har Har Mahadev (Lyrical Video): Sachet Tandon...,"T-Series Presents ""Har Har Mahadev (Lyrical Vi...","[hindi songs 2023, hindi songs new, bollywood ...",2023-08-20T05:30:06Z,101950,6230,0,434,PT3M37S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19340,aouxczBof58,T-Series,ROLA PE GAYA FULL SONG| PATIALA HOUSE | AKSHA...,"Watch the song ""Rola Pe Gaya"" from the movie '...","[Rola Pe Gaya, Anushka Sharma, Patiala House, ...",2011-01-20T13:34:53Z,636734,943,0,18,PT1M31S,hd,false
19341,NPhY3g0WE8c,T-Series,Patiala House-Official Trailer II,Patiala house upcoming movie official trailer,"[new songs, official trailer, Songs, Dimple Ka...",2011-01-13T11:27:42Z,86841,1084,0,4,PT3M32S,sd,false
19342,ZVYTLHLpgAw,T-Series,Kyun Mein Jagoon Song (Patiala House ) | Aksha...,"""Here's presenting the song """"Kyun Main Jagoon...","[film songs, songs, bollywood song, anushka sh...",2011-01-07T09:36:16Z,364995,1865,0,30,PT1M46S,sd,false
19343,hArZnPWxb4A,T-Series,Laung Da Lashkara (Patiala House) Full Song | ...,"Watch the full video song ""Laung Da Lashkara"" ...","[laungda lashkara, laung da lashkara, akshay k...",2011-01-03T07:10:25Z,13663672,67331,0,2483,PT2M51S,hd,false


In [ ]:
# exporting to excel files

df1.to_excel("YT_Tseries.xlsx")